In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn

In [10]:
df = pd.read_json(r"C:\Users\Niranjan\Desktop\Data Science\datasets\Supervised learning potpourri\Amazon_Instant_Video_5.json", orient='columns', lines=True)
df.head(5)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,B000H00VBQ,"[0, 0]",2,I had big expectations because I love English ...,"05 3, 2014",A11N155CW1UV02,AdrianaM,A little bit boring for me,1399075200
1,B000H00VBQ,"[0, 0]",5,I highly recommend this series. It is a must f...,"09 3, 2012",A3BC8O2KCL29V2,Carol T,Excellent Grown Up TV,1346630400
2,B000H00VBQ,"[0, 1]",1,This one is a real snoozer. Don't believe anyt...,"10 16, 2013",A60D5HQFOTSOM,"Daniel Cooper ""dancoopermedia""",Way too boring for me,1381881600
3,B000H00VBQ,"[0, 0]",4,Mysteries are interesting. The tension betwee...,"10 30, 2013",A1RJPIGRSNX4PW,"J. Kaplan ""JJ""",Robson Green is mesmerizing,1383091200
4,B000H00VBQ,"[1, 1]",5,"This show always is excellent, as far as briti...","02 11, 2009",A16XRPF40679KG,Michael Dobey,Robson green and great writing,1234310400


In [11]:
df.shape

(37126, 9)

In [13]:
#3and below --ve
df['ReviewType'] = np.where(df.overall>=3,1,0)

In [14]:
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,ReviewType
0,B000H00VBQ,"[0, 0]",2,I had big expectations because I love English ...,"05 3, 2014",A11N155CW1UV02,AdrianaM,A little bit boring for me,1399075200,0
1,B000H00VBQ,"[0, 0]",5,I highly recommend this series. It is a must f...,"09 3, 2012",A3BC8O2KCL29V2,Carol T,Excellent Grown Up TV,1346630400,1
2,B000H00VBQ,"[0, 1]",1,This one is a real snoozer. Don't believe anyt...,"10 16, 2013",A60D5HQFOTSOM,"Daniel Cooper ""dancoopermedia""",Way too boring for me,1381881600,0
3,B000H00VBQ,"[0, 0]",4,Mysteries are interesting. The tension betwee...,"10 30, 2013",A1RJPIGRSNX4PW,"J. Kaplan ""JJ""",Robson Green is mesmerizing,1383091200,1
4,B000H00VBQ,"[1, 1]",5,"This show always is excellent, as far as briti...","02 11, 2009",A16XRPF40679KG,Michael Dobey,Robson green and great writing,1234310400,1


In [35]:
x = pd.Series(''.join(df['reviewText']).lower().split()).value_counts()[:1000]
print(x)

the          182566
and          101211
a             86892
to            82963
of            76788
is            63091
i             52205
in            43411
it            41808
this          39535
that          36494
for           27525
with          26302
but           25376
as            24014
was           22636
on            21494
are           21460
you           18795
not           18208
have          17116
show          16915
they          15675
be            15486
like          13881
all           12606
season        12416
so            12398
one           12246
his           11998
              ...  
aliens          306
budget          306
art             305
weird           304
question        304
acted           304
people.         304
subject         304
watch,          304
piece           303
done.           303
tells           302
none            302
sister          301
it!             301
becoming        300
hollywood       300
deep            300
younger         299


In [45]:
x.to_csv(r'C:\Users\Niranjan\Desktop\Data Science\datasets\Supervised learning potpourri\CommonWords.csv', index=True)

c:\users\niranjan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [77]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

X = df['reviewText']
Y = df['ReviewType']
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=20)

stop_words = ['in', 'of', 'at', 'a', 'the']
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 3), stop_words=stop_words)
ngram_vectorizer.fit(X)
X_vectorized = ngram_vectorizer.transform(X)


In [78]:
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, Y, train_size = 0.70)
for c in [0.001, 0.005, 0.01, 0.05, 0.1]:
    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test, svm.predict(X_test))))
    

final = LinearSVC(C=0.1)
final.fit(X_train, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(Y, final.predict(X_vectorized)))

Accuracy for C=0.001: 0.9178488058897468
Accuracy for C=0.005: 0.9291614293409948
Accuracy for C=0.01: 0.931136649308673
Accuracy for C=0.05: 0.934368827437601
Accuracy for C=0.1: 0.9353564374214401
Final Accuracy: 0.9804988417820395


In [76]:
print(df['ReviewType'].ravel())
print(final.predict(X_vectorized))

[0 1 0 ... 1 1 1]
[1 1 0 ... 1 1 1]


In [87]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.linear_model import LogisticRegression
for c in [0.001, 0.005, 0.01, 0.05, 0.1]:
    svm = LogisticRegression(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test, svm.predict(X_test))))
    

finalLR = LogisticRegression(C=0.1)
finalLR.fit(X_train, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(Y, finalLR.predict(X_vectorized)))

Accuracy for C=0.001: 0.9048303106482313
Accuracy for C=0.005: 0.9144370623092117
Accuracy for C=0.01: 0.9192853295026037
Accuracy for C=0.05: 0.9293409947926019
Accuracy for C=0.1: 0.9315855629376908
Final Accuracy: 0.9756774228303615
